# Analýza sentimentu v příspěvcích a komentářích na sociálních sítích
Tým: Ondřej Chmelík, Martin Vondráček, mentor Ing. Jan Lehečka, Ph.D.

## Abstrakt
Cílem tohoto projektu je analyzovat komentáře ze sociálních sítí a klasifikovat jejich rizikovost (jak pravděpodobné je, že daný komentář vyvolá negativní reakce v odpovědích na daný komentář). Pro klasifikaci použijeme moderní transformerové jazykové modely. Výsledné natrénované modely následně otestujeme na příspěvcích na sociálních sítích (Twitter/X, Reddit). Tento proces může dobře posloužit k vyhledání “rizikových faktorů” – může se jednat o konkrétní témata vyvolávající nevhodné a agresivní reakce nebo detekci toxicity ve větších komunitách (například v subredditech na Redditu).

Projekt má široké využití v praxi – například při moderování online diskuzí a filtrování nevhodných příspěvků a komentářů, kde může posloužit při předvídání nevhodných odezev a jejich předejití (například předběžné upozornění pro moderátory diskuzí, blokování vláken atp.). Naším cílem je nejen vytvořit přesný klasifikátor, ale také pochopit jeho možnosti a omezení v různých kontextech.

## Postup při řešení problematiky
Pro zpracování projektu jsme použili následující zdroje:

Modely:
- BERT Large Uncased

Datasety:
- Twitter Emotions Classification Dataset
- GoEmotions

Původně jsme zamýšleli využít dataset GoEmotions ke kompletnímu natrénování finálního modelu, avšak vzhledem k nespolehlivosti labelů v tomto datasetu (až 30% vzorků má chybné labely) jsme byli nuceni změnit strategii a využít komplexnějšího přístupu. Nejprve jsme natrénovali normální klasifikační model na rozpoznávání emocí na Twitter Emotions Classification datasetu a následně tento model využili k přelabelování datasetu GoEmotions, který již obsahuje stromovou strukturu. Na té jsme pak natrénovali finální klasifikační model, který už vyhodnocuje rizikovost samotných komentářů (jak pravděpodobné je, že daný komentář vyvolá negativní reakce).

### Trénování základního modelu
Program pro trénování základního modelu je uložen v notebooku [base_model.ipynb](base_model.ipynb). Zkoušeli jsme různé parametry pro trénování (změny learning rate, změny batch size, použití pouhé části datasetu) a zároveň i porovnali výsledek trénování pro celý model a pouze pro klasifikační hlavičku. Trénování celého modelu nakonec dosáhlo lepších výsledků na validačním datasetu, tudíž jsme tento model použili při dalším postupu (složka emotion_model_biglr_full).

Základní model lze manuálně otestovat v notebooku [base_test_manual.ipynb](base_test_manual.ipynb)

### Trénování finálního modelu
Základní model jsme následně využili v notebooku [goemotions_relabel.ipynb](goemotions_relabel.ipynb) k přelabelování datasetu GoEmotions pro naše potřeby. V notebooku [parent_model.ipynb](parent_model.ipynb) jsme následně provedli samotné zpracování nově olabelovaného datasetu a natrénování finálního modelu. Stromová struktura v datasetu GoEmotions se ukázala jako nedostatečná a výsledná užitečná část datasetu byla příliš malá pro naše účely (~200 vzorků), tudíž jsme dataset uměle rozšířili.

Pro trénování jsme opět vyzkoušeli různé parametry. Původně měl model problém s overfittingem, což jsme vyřešili přidáním weight decay. Natrénovaný model dosáhl na validačním datasetu přesnosti mezi 75-80 % a F1 skóre kolem 0.8, jak lze vidět v přiložené tabulce níže.

|Epoch|Training loss|Validation loss|Accuracy|F1|Precision|Recall|
|---|---|---|---|---|---|---|
|1|0.672|0.648|0.605|0.752|0.602|1.000|
|2|0.579|0.573|0.732|0.799|0.724|0.891|
|3|0.448|0.530|0.745|0.797|0.761|0.837|
|4|0.548|0.514|0.752|0.793|0.789|0.798|
|5|0.417|0.509|0.754|0.792|0.802|0.782|
|6|0.603|0.507|0.762|0.804|0.795|0.813|
|7|0.395|0.508|0.759|0.798|0.800|0.796|
|8|0.443|0.507|0.765|0.806|0.796|0.817|
|9|0.458|0.508|0.763|0.803|0.796|0.811|
|10|0.409|0.508|0.765|0.806|0.797|0.815|

## Příklady komentářů

### Příklady klasifikace při manuálním testování různých vstupů
|"I love dinner."||
|---|---|
|Třída 0 (Normální komentář)|87.24%|
|Třída 1 (Rizikový komentář)|12.76%|

<br>

|"You’re lucky it didn’t get disabled after 10 attempts"||
|---|---|
|Třída 0 (Normální komentář)|50.12%|
|Třída 1 (Rizikový komentář)|49.88%|

<br>

|"Leave the house. By staying there you can only make it worse."||
|---|---|
|Třída 0 (Normální komentář)|13.25%|
|Třída 1 (Rizikový komentář)|86.75%|

<br>

|"Don't stick your nose into politics again"||
|---|---|
|Třída 0 (Normální komentář)|11.30%|
|Třída 1 (Rizikový komentář)|88.70%|

<br>

|"You have already beaten 99 percent of the people, keep going"||
|---|---|
|Třída 0 (Normální komentář)|37.76%|
|Třída 1 (Rizikový komentář)|62.24%|

### Příklady nesprávně klasifikovaných příspěvků z datasetu
|"Married at first sight!" (label 0)||
|---|---|
|Třída 0 (Normální komentář)|17.66%|
|Třída 1 (Rizikový komentář)|82.34%|

<br>

|"Oh easy leader of the free world" (label 1)||
|---|---|
|Třída 0 (Normální komentář)|73.33%|
|Třída 1 (Rizikový komentář)|26.67%|


Níže je k dispozici kód pro načtení finálního modelu a jeho manuální otestování. Pro relativně jasné vstupy vrátil model očekávané a správné výstupy, každopádně kvůli více stupňům trénování v rámci projektu a omezení ze strany dataestů očekáváme vetěí frekvenci chybných výsledků u komplexnějších vstupů.

In [1]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

model_path = "./parent_model"  # or wherever you saved it

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

model.eval()

c:\Users\Ondra\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
text = input()
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    softmax_outputs = torch.softmax(logits, dim=1)
    predicted_class_id = softmax_outputs.argmax().item()

    probs = softmax_outputs[0]
    percentages = (probs * 100).tolist()

    for i, p in enumerate(percentages):
        print(f"Class {i} ({['Normal comment', 'Controversial comment'][i]}): {p:.2f}%")

Class 0 (Normal comment): 73.33%
Class 1 (Controversial comment): 26.67%
